In [1]:
# Load Modules
import os
import numpy as np
import json
import math

# import seaborn as sns
# sns.set()

#from tqdm.notebook import tqdm

# Pytorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision

# cuda setup, set seed for reproducability 
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
set_seed(41)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print(f"Using device: {device}")


# YOU NEED THIS TO LOAD PyTorch Lightning I DON"T KNOW WHY

from jupyter_client.manager import KernelManager

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Set torch dtype to float64
torch.set_default_dtype(torch.float64)


/home/tdmonkman/anaconda3/envs/please_work/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tdmonkman/anaconda3/envs/please_work/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Using device: cpu


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline



# Set data path

In [3]:
# Modules for loading data, set data pathimport urllib
DATASET_PATH = "cloudimages1M"
CHECKPOINT_PATH = os.getcwd()
DRIVE_PATH = "."

# Import custom npy loader
import sys
sys.path.append(f"{DRIVE_PATH}/")
from import_npy import npy_loader



# Look at example pic


In [4]:
img_index = 3
group = 1

file = os.listdir(f"{DRIVE_PATH}/{DATASET_PATH}/{group}")[img_index]
img_example = np.load(f"{DRIVE_PATH}/{DATASET_PATH}/{group}/{file}",allow_pickle=True)

fig, axs = plt.subplots(2,3,figsize=(20,10))

axs[0,0].pcolor(img_example[:,:,0],cmap="plasma")
axs[0,1].pcolor(img_example[:,:,1],cmap="plasma")
axs[0,2].pcolor(img_example[:,:,2],cmap="plasma")

axs[1,0].pcolor(img_example[:,:,3],cmap="plasma")
axs[1,1].pcolor(img_example[:,:,4],cmap="plasma")
axs[1,2].pcolor(img_example[:,:,5],cmap="plasma")

plt.show()
plt.close()

FileNotFoundError: [Errno 2] No such file or directory: './cloudimages1M/1'

# Define the DataLoaders

In [ ]:
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets

# Here we use our custom function imported from the drive directory (import_npy.py)
AICCA_data = datasets.DatasetFolder(root=f"{DRIVE_PATH}/{DATASET_PATH}",
                                    loader=npy_loader,
                                    extensions=['.npy'])
    

print(AICCA_data)
print("classes")
print(AICCA_data.classes)
print("class_dict")
print(AICCA_data.class_to_idx)

# Data loader works her
# Split into train, validation, and test data
train_length = int(0.7*len(AICCA_data))
validation_length = int(0.2*len(AICCA_data))
test_length = len(AICCA_data) - train_length - validation_length

# Split
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(AICCA_data, (train_length, validation_length, test_length))

# Verify size of datasets
print(f"size train_dataset: {len(train_dataset)}")
print(f"size validation_dataset: {len(validation_dataset)}")
print(f"size test_dataset: {len(test_dataset)}")

# 
train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=False, pin_memory=True,num_workers=1)
val_loader = data.DataLoader(validation_dataset, batch_size=128, shuffle=False, drop_last=False, pin_memory=True,num_workers=1)
test_loader = data.DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False, pin_memory=True,num_workers=1)

In [ ]:
test_dataset

# Try randomly sampling images

In [ ]:
img, label = train_dataset[0][0], train_dataset[0][1]
#print(f"Image tensor:\n{img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

NUM_IMAGES = 5
idx = 3

cloud_images = torch.stack([test_dataset[idx][0] for idx in np.random.randint(0,high=test_length,size=NUM_IMAGES)], dim=0)
img_grid = torchvision.utils.make_grid(torch.permute(cloud_images[:,:,:,[0,1,2]],(0,3,1,2)), nrow=NUM_IMAGES, normalize=True, pad_value=0.9)
img_grid = img_grid.permute(1,2,0)

plt.figure(figsize=(10*NUM_IMAGES,8))
plt.imshow(img_grid)
plt.axis('off')
#plt.savefig("eval_figures/example_patches0.png",transparent=True)
plt.show()
plt.close()

In [ ]:
128*128/(8*8)

# Transform images to patches

In [ ]:
# Function for preprocessing the images into patches
def imgs_to_patches(imgs, patch_size, flatten_channels=True):
    """
    Inputs
    ------
    imgs: torch.Tensor containing the images of shape (Num Images, Channels, Height, Width)
    patch_size: 
    flatten_channels: False
    
    Outputs
    -------
    imgs: 
    """
    # Reshape the image tensor to shape (image, channel, height, width) 
    imgs = imgs.permute(0,3,1,2)
    B, C, H, W = imgs.shape
    # Divide images into patches
    imgs = imgs.reshape(B, C, H//patch_size, patch_size, W//patch_size, patch_size)
    # Reshape images to shape (image, patch, channel, height, width)
    imgs = imgs.permute(0,2,4,1,3,5)
    imgs = imgs.flatten(1,2)
    # You can flatten the patches into a "feature vector" if you wouldlike
    if flatten_channels:
        imgs = imgs.flatten(2,4)
    return imgs
    
    
cloud_images = torch.stack([test_dataset[idx][0][:,:,:3] for idx in np.random.randint(0,high=test_length,size=NUM_IMAGES)], dim=0)
img_patches = imgs_to_patches(cloud_images, 16)

# Plot examples
fig, ax = plt.subplots(cloud_images.shape[0], 1, figsize=(20,4))
fig.suptitle("Images as input sequences of patches")
for i in range(cloud_images.shape[0]):
  img_grid = torchvision.utils.make_grid(img_patches[i], nrow=64, normalize=True, pad_value=0.9)
  img_grid = img_grid.permute(1,2,0)
  ax[i].imshow(img_grid)
  ax[i].axis('off')

plt.show()
plt.close()

In [ ]:
print(128/4)
print(128/8)
print(128/16)
print(128/32)
print(128/64)

# Attention Block Prototype
see https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial15/Vision_Transformer.html

In [11]:
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """
        Inputs
        ------
        embed_dim: Dimensionality of input and attention feature vectors
        hidden_dim: Dimensionality of hidden layer in feed-forward network
                    (usually 2 to 4x larger than embed_dim)
        num_heads: Number of heads to use in the Multi-Head Attention block
        dropout: Amount of dropout to apply in the feed-forward network
        """
        super().__init__() # super() is a function that allows you to initialize 
                           # attributes from the parent class
        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        # nn.LayerNorm applies layer normalization over a mini-batch of inputs as
        # described in the AttentionBlock paper. Uses the expectation value and 
        # mean to calculate them over the last D-dimensions where D is the 
        # dimension of the 'normalized_shape.'
        
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        # nn.MultiheadAttention applies the multiheaded attention layer from the
        # Attention paper. 'embed_dim' and 'num_heads' are self-explanatory.
        # During training, dropout randomly zeros some of the elements of the 
        # input tensor with probability p using samples from a Bernoulli 
        # distribution. Each channel will be zeroed out independently on
        # every forward call. This is an effective technique for regularization
        # and preventing the co-adaption of neurons.
        
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        # take the norm again
        
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout),
        )
        
    def forward(self, x):
        # run the attention block
        input_x = self.layer_norm_1(x)
        x = x + self.attn(input_x, input_x, input_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x

## Build the Vision Transformer
Besides the transformer, we need a couple more modules:
*    A linear projection layer that maps the input patches to a feature vector of larger size. It is implemented by a linear layer that takes each MxM patch independently as input.
*    A classification token that is added to the input sequence. We will use the output feature vector of the classification token (CLS token) for determining the classification prediction.
*    Learnable positional encodings that are added to the tokens before being processed by the transformer. Those are needed to learn position-dependent information, and convert the set to a sequence. SINCE WE ARE WORKING WITH A FIXED RESOLUTION, WE CAN LEARN THE POSITIONAL ENCODINGS INSTEAD OF HAVING THE PATTERN OF SINE AND COSINE FUNCTIONS!
*    An MLP head that takes the output feature vector of the CLS token, and maps it to a classification prediction. This is usually inmplemented by a small feed-forward network or even a single linear layer. 



In [12]:
class VisionTransformer(nn.Module):
    
    def __init__(self, embed_dim, hidden_dim, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, dropout=0.0):
        """
        Inputs
        ------
        embed_dim: Dimensionality of the input feature vectors to the Transformer
        hidden_dim: Dimensionality of the hidden layer in the feed-forward networks
                     within the Transformer
        num_channels: Number of channels of the input (3 for RGB)
        num_heads: Number of heads to use in the Multi-Head Attention block
        num_layers: Number of layers to use in the Transformer
        num_classes: Number of classes to predict
        patch_size: Number of pixels that the patches have per dimension
        num_patches: Maximum number of patches an image can have
        dropout: Amount of dropout to apply in the feed-forward network and
                  on the input encoding
        
        Dependencies
        ------------
        
        """
        super().__init__()
        
        self.patch_size = patch_size
        
        # Layers/Networks
        self.input_layer = nn.Linear(num_channels*(patch_size**2), embed_dim)
        self.transformer = nn.Sequential(*[AttentionBlock(embed_dim, hidden_dim, num_heads, dropout=dropout) for layer in range(num_layers)])
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )
        self.dropout = nn.Dropout(dropout)
        
        # Parameters/Embeddings
        self.cls_token = nn.Parameter(torch.randn(1,1,embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1,1+num_patches,embed_dim))
        
    def forward(self, img):
        # Preprocess input
        img = imgs_to_patches(img, self.patch_size)
        B, T, _ = img.shape
        img = self.input_layer(img)
        
        # Add CLS token and positional encoding
        cls_token = self.cls_token.repeat(B, 1, 1)
        img = torch.cat([cls_token, img], dim=1)
        img = img + self.pos_embedding[:,:T+1]
        
        # Apply Transformer
        img = self.dropout(img)
        img = img.transpose(0,1)
        img = self.transformer(img)
        
        # Perform classification prediction
        cls = img[0]
        out = self.mlp_head(cls)
        return out
    
    


# Decoderclass Decoder(nn.Module):


In [ ]:
class Conv_Decoder(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, dropout=0.0):

    def __init__(self, num_input_channels: int, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU):
        """
        Args:
           num_input_channels : Number of channels of the image to reconstruct. For AICCA, this parameter is 6
           base_channel_size : Number of channels we use in the last convolutional layers. Early layers might use a duplicate of it.
           latent_dim : Dimensionality of latent representation z (we use 43 cases)
           act_fn : Activation function used throughout the decoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.linear = nn.Sequential(nn.Linear(latent_dim, 2 * 16 * c_hid), act_fn())
        self.net = nn.Sequential(
            nn.ConvTranspose2d(
                2 * c_hid, 2 * c_hid, kernel_size=3, output_padding=1, padding=1, stride=2
            ),  # 4x4 => 8x8
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(2 * c_hid, c_hid, kernel_size=3, output_padding=1, padding=1, stride=2),  # 8x8 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(
                c_hid, num_input_channels, kernel_size=3, output_padding=1, padding=1, stride=2
            ),  # 16x16 => 32x32
            nn.Tanh(),  # The input images is scaled between -1 and 1, hence the output has to be bounded as well
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], -1, 4, 4)
        x = self.net(x)
        return x
    
    
    

# ViT Autoencoder

In [13]:
from lightning_fabric.utilities import optimizer

class ViT_Autoencoder(pl.LightningModule):

    def __init__(self, model_kwargs, lr):
        super().__init__()
        self.save_hyperparameters()
        self.forward_model = VisionTransformer(**model_kwargs)
        self.decoder = Conv_Decoder(**model_kwargs)

    def forward(self, x):
        """
        x: patches
        Forward function is a ViT encoder
        Decoder function is just a simple convolutional filter
        """
        y = self.forward_model(x)
        x_hat = self.decoder(y)
        return x_hat
    
    def _get_reconstruction_loss(self, batch):
        """
        Given a batch of images, this function returns the reconstruction loss (using MSE here)
        """
        x, _ = batch
        x_hat = self.forward(x)
        loss = F.mse_loss(x, x_hat, reduction="None")
        loss = loss.sum(dim=[1,2,3]).mean(dim=[0])
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)
        return [optimizer], [lr_scheduler]

    def training_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch, mode="train")
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        self._get_reconstruction_loss(batch, mode="val")
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode="test")
        self.log("test_loss", loss)




In [14]:
def train_model(training_checkpoint_path,**kwargs):
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, "ViT"),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=180,
                         callbacks=[ModelCheckpoint(save_weights_only=True, verbose=True, save_last=True, mode="max", monitor="val_acc", every_n_train_steps=100),
                                    LearningRateMonitor("epoch")])
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    print(f"cwd: {os.getcwd()}")
    print(f"CHECKPOINT_PATH: {CHECKPOINT_PATH}")
    pretrained_filename = os.path.join(CHECKPOINT_PATH, "ViT.ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        model = ViT.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
    
    elif os.path.isfile(CHECKPOINT_PATH):
        print(f"Found checkpoint at {CHECKPOINT_PATH}")
        model = ViT.load_from_checkpoint(CHECKPOINT_PATH) # Load best checkpoint after training
        trainer.fit(model, train_loader, val_loader)
        model = ViT.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    else:
        print("Running from scratch")
        pl.seed_everything(42) # To be reproducable
        model = ViT(**kwargs)
        trainer.fit(model, train_loader, val_loader)
        model = ViT.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"test": test_result[0]["test_acc"], "val": val_result[0]["test_acc"]}

    return model, result

In [15]:
CHECKPOINT_PATH

'/scratch/midway2/tdmonkman/AICCA_proj'

# Train the model here

In [16]:
model, results =     train_model( '/scratch/midway2/tdmonkman/AICCA_proj', model_kwargs={
                                    'embed_dim': 256,
                                    'hidden_dim': 512,
                                    'num_heads': 8,
                                    'num_layers': 1,
                                    'patch_size': 16,
                                    'num_channels': 6,
                                    'num_patches': 64,
                                    'num_classes': 42,
                                    'dropout': 0.2
                                }, lr=3e-4,)


print("ViT results", results)

/home/tdmonkman/anaconda3/envs/please_work/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/tdmonkman/anaconda3/envs/please_work/lib/pytho ...
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


cwd: /scratch/midway2/tdmonkman/AICCA_proj
CHECKPOINT_PATH: /scratch/midway2/tdmonkman/AICCA_proj
Running from scratch



KeyboardInterrupt

